In [1]:
# BETA TEST 
# Import dependencies
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import pandas as pd
from datetime import date
import random
import time
import schedule
from spotifire_id import *
import os
import requests
import sys

# Initial setup
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=c_id, # Import spotify authentication
                                            client_secret=s_id,
                                            redirect_uri="http://localhost:1410/",
                                            scope=("user-library-read","playlist-modify-private")))

# Define me
me = my_id

# Playlist id's
playlist1_uid=p1_uid
playlist2_uid=p2_uid
playlist3_uid=p3_uid
playlist4_uid=p4_uid
playlist_combo_uid=pc_uid
playlist_combo2_uid=pc2_uid

# Download spotifire google sheet
def getGoogleSheet(spreadsheet_id, outDir, outFile):
    
    url = f'https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv'
    response = requests.get(url)
    if response.status_code == 200:
        filepath = os.path.join(outDir, outFile)
        with open(filepath, 'wb') as f:
            f.write(response.content)
            print('CSV file saved to: {}'.format(filepath))
    else:
        print(f'Error downloading Google Sheet: {response.status_code}')

# Add songs to playlists
def add_songs_to_playlist(playlist_id, combo_id, songs):
    split_index = 0
    while split_index < len(songs):
        split = songs[split_index:split_index + 99]
        if playlist_id == playlist1_uid or playlist_id == playlist2_uid:
            if split_index == 0:
                sp.playlist_replace_items(playlist_id=playlist_id, items=split)
                sp.playlist_replace_items(playlist_id=combo_id, items=split)
            else:
                sp.playlist_add_items(playlist_id=playlist_id, items=split, position=None)
                sp.playlist_add_items(playlist_id=combo_id, items=split, position=None)
        else:
            if split_index == 0:
                sp.playlist_replace_items(playlist_id=playlist_id, items=split)
                sp.playlist_add_items(playlist_id=combo_id, items=split)
            else:
                sp.playlist_add_items(playlist_id=playlist_id, items=split, position=None)
                sp.playlist_add_items(playlist_id=combo_id, items=split, position=None)
        time.sleep(6)
        split_index += 99

# Main function
def spotifire():

    # Download spotifire and spotifire_schedule
    outDir = 'Resources/'
    os.makedirs(outDir, exist_ok = True)
    filepath = getGoogleSheet(s_csv, outDir, "SpotiFire.csv")
    os.makedirs(outDir, exist_ok = True)
    filepath = getGoogleSheet(ss_csv, outDir, "SpotiFire_Schedule.csv")

    # Extract date from calender
    today = date.today()
    today = str(today)
    year, month, day = map(int, today.split("-"))
    today = f"{year},{month},{day}"
    print(f"Creating playlist for {today}")

    # Manual entry
    # today = '2024,6,9'
    
    # Key words used to filter out holiday music
    holiday_key_words = ['Christmas','christmas','Santa','santa']

    # Import schedule
    schedule = pd.read_csv('Resources/SpotiFire_Schedule.csv')

    # Extract todays schedule
    today_schedule = schedule.loc[schedule['Date'] == today]
    today_schedule = today_schedule.reset_index(drop=True)

    line_list = []
    line_schedules = []

    # Break schedule into lines
    line_list = []
    line_schedules = []
    for line_num in range(1, 5):
        line_schedule = today_schedule.loc[0, [f'Line {line_num} ({i})' for i in range(1, 10)]].values.tolist()
        line_schedules.append(line_schedule)
        line_list.extend(line_schedule)


    # Import music choices
    choices = pd.read_csv("Resources/SpotiFire.csv")

    # Lists to hold artists and songs
    artists = []
    songs = []
    line1_songs = []
    line2_songs = []
    line3_songs = []
    line4_songs = []

    # Loop to collect songs for each formulator
    for i in range(len(line_list)):
        artists = [] # Reset artist list
        songs = [] # Reset song list
        formulator_choices = choices.loc[line_list[i] == choices['id']] # Select row from spotifire where ID matches the schedule
        print("--------------------------------------------------------------------")
        print("--------------------------------------------------------------------")
        print(f"Collecting artists for {line_list[i]}")
        
        # Check if cell is empty in spotifire_schedule
        if formulator_choices.empty:
            continue  # Skip to the next iteration if it's empty
        
        # Select up to ten artists from formulator's row
        for t in range(10):
            artists.append(formulator_choices.iloc[0,t+1])
            print(f"Artist choice {t}:{formulator_choices.iloc[0,t+1]}")
        
        # Clean the artist list
        artists_clean = pd.DataFrame(artists) # Convert to dataframe
        artists_clean = artists_clean.dropna() # Drop nan
        artists_clean = artists_clean.reset_index(drop=True) # Reset index

        # Loop through artist list
        for t in range(len(artists_clean)):
            artist_name = artists_clean[0][t]
            search_results = sp.search(q='artist:' + artist_name, type='artist', limit=1)
            if search_results['artists']['total'] == 0: # If no artists found
                print(f"No results for {artist_name}") # Print message
                continue # Skip
            print(f"Searching {artist_name}'s related artists")
            time.sleep(6)

            # Collect URI's
            artist_uri = search_results['artists']['items'][0]['uri']
            related_artist_search_results = sp.artist_related_artists(artist_uri) # Search related artists
            related_artist_uri1 = related_artist_search_results['artists'][0]['uri']
            related_artist_uri2 = related_artist_search_results['artists'][1]['uri']
            related_artist_uri3 = related_artist_search_results['artists'][2]['uri']

            # Collect top tracks for each artist
            artist_top_songs = sp.artist_top_tracks(artist_uri,country='US')
            related_artist_top_songs1 = sp.artist_top_tracks(related_artist_uri1, country='US')
            print(f'Related artist 1: {related_artist_search_results["artists"][0]["name"]}')
            time.sleep(6)
            related_artist_top_songs2 = sp.artist_top_tracks(related_artist_uri2, country='US')
            print(f'Related artist 2: {related_artist_search_results["artists"][1]["name"]}')
            time.sleep(6)
            related_artist_top_songs3 = sp.artist_top_tracks(related_artist_uri3, country='US')
            print(f'Related artist 3: {related_artist_search_results["artists"][2]["name"]}')
            time.sleep(6)

            # Randomly select one song from each related artist
            related_artist_top_songs_sample1 = random.sample(related_artist_top_songs1['tracks'],1)
            related_artist_top_songs_sample2 = random.sample(related_artist_top_songs2['tracks'],1)
            related_artist_top_songs_sample3 = random.sample(related_artist_top_songs3['tracks'],1)

            # Filter out explicit and holiday tracks
            sample_lists = [related_artist_top_songs_sample1, related_artist_top_songs_sample2, related_artist_top_songs_sample3] # List of sampled tracks
            for sample in sample_lists:
                for j in range(len(sample)):
                    if sample[j]['name'] in holiday_key_words: # Check for holiday track
                        print(f"Track '{sample[j]['name']}' by {sample[j]['artists'][0]['name']} removed for Holiday status.")
                        continue
                    if sample[j]['explicit'] == False: # Check for explicit
                        print(f"Track '{sample[j]['name']}' by {sample[j]['artists'][0]['name']} added to sample pool.")
                        songs.append(sample[j]['uri'])
                    else:
                        print(f"Track '{sample[j]['name']}' by {sample[j]['artists'][0]['name']} removed for explicit rating.")

            for j in range(len(artist_top_songs['tracks'])):
                if artist_top_songs['tracks'][j]['name'] in holiday_key_words: # Check for holiday track
                    print(f"Track '{artist_top_songs['tracks'][j]['name']}' by {artist_top_songs['tracks'][j]['artists'][0]['name']} removed for Holiday status.")
                    continue
                if artist_top_songs['tracks'][j]['explicit'] == False: # Check for explicit
                    print(f"Track '{artist_top_songs['tracks'][j]['name']}' by {artist_top_songs['tracks'][j]['artists'][0]['name']} added to sample pool.")
                    songs.append(artist_top_songs['tracks'][j]['uri'])
                else:
                    print(f"Track '{artist_top_songs['tracks'][j]['name']}' by {artist_top_songs['tracks'][j]['artists'][0]['name']} removed for explicit rating.")

        # If song count is below 30
        if len(songs) < 30:
            songs_sample = random.sample(songs, len(songs))
            if len(songs) > 2:
                filler_tracks = random.sample(related_artist_top_songs1['tracks'],3) # Grab 3 filler tracks from related artist 1
                for j in range(len(filler_tracks)):
                    if filler_tracks[j]['explicit'] == False: # Check if songs are explicit
                        songs_sample.append(filler_tracks[j]['uri'])
                        print(f"Track '{filler_tracks[j]['name']}' by {filler_tracks[j]['artists'][0]['name']} added to sample pool.")
                    else:
                        print(f"Track '{filler_tracks[j]['name']}' by {filler_tracks[j]['artists'][0]['name']} removed for explicit rating.")
                        
        # If song count is 30 or greater, take a random sample of 30 songs and add to playlist         
        else: 
            songs_sample = random.sample(songs,30)
        if line_list[i] in line_schedules[0]:
            line1_songs.extend(songs_sample)
            print(f"Formulator {line_list[i]} is adding {len(songs_sample)} songs to line 1")
        if line_list[i] in line_schedules[1]:
            line2_songs.extend(songs_sample)
            print(f"Formulator {line_list[i]} is adding {len(songs_sample)} songs to line 2")
        if line_list[i] in line_schedules[2]:
            line3_songs.extend(songs_sample)
            print(f"Formulator {line_list[i]} is adding {len(songs_sample)} songs to line 3")
        if line_list[i] in line_schedules[3]:
            line4_songs.extend(songs_sample)
            print(f"Formulator {line_list[i]} is adding {len(songs_sample)} songs to line 4")

    # Remove duplicates
    line1_songs = list(set(line1_songs))
    line2_songs = list(set(line2_songs))
    line3_songs = list(set(line3_songs))
    line4_songs = list(set(line4_songs))

    # Add playlists to spotify
    playlists = [(playlist1_uid, playlist_combo_uid), (playlist2_uid, playlist_combo2_uid), (playlist3_uid, playlist_combo_uid), (playlist4_uid, playlist_combo2_uid)]
    song_lists = [line1_songs, line2_songs, line3_songs, line4_songs]
    for (playlist_id, combo_id), songs in zip(playlists, song_lists):
        if len(songs) > 0:
            add_songs_to_playlist(playlist_id, combo_id, songs)
        if len(songs) > 99:
            add_songs_to_playlist(playlist_id, combo_id, songs[100:])
        if len(songs) > 199:
            add_songs_to_playlist(playlist_id, combo_id, songs[200:])
        if len(songs) > 299:
            add_songs_to_playlist(playlist_id, combo_id, songs[300:])

    # Update playlist descriptions
    for i in range(4):
        playlist_uid = globals()[f"playlist{i+1}_uid"]
        schedule = line_schedules[i]
        description = f"Date: {today[5:]},{today[2:4]}. Formulated for: {', '.join(schedule)}."
        sp.user_playlist_change_details(me, playlist_uid, description=description)
    sp.user_playlist_change_details(me, playlist_combo_uid, description=f"Date: {today[5:]},{today[2:4]}. Formulated for: {', '.join(line_schedules[0])}, {', '.join(line_schedules[2])}.")
    sp.user_playlist_change_details(me, playlist_combo2_uid, description=f"Date: {today[5:]},{today[2:4]}. Formulated for: {', '.join(line_schedules[1])}, {', '.join(line_schedules[3])}.")

schedule.every().day.at("05:00").do(spotifire)
while True:
    schedule.run_pending()
    time.sleep(60)

CSV file saved to: Resources/SpotiFire.csv
CSV file saved to: Resources/SpotiFire_Schedule.csv
Creating playlist for 2024,6,8
--------------------------------------------------------------------
--------------------------------------------------------------------
Artist choice 0:Don Broco
Artist choice 1:Ghost
Artist choice 2:NF
Artist choice 3:Loveless
Artist choice 4:Siamese
Artist choice 5:nan
Artist choice 6:nan
Artist choice 7:nan
Artist choice 8:nan
Artist choice 9:nan
Searching Don Broco's related artists
Related artist 1: Lower Than Atlantis
Related artist 2: Mallory Knox
Related artist 3: Deaf Havana
Track 'Another Sad Song' by Lower Than Atlantis added to sample pool.
Track 'Sugar' by Mallory Knox added to sample pool.
Track 'Everybody's Dancing and I Want to Die' by Deaf Havana added to sample pool.
Track 'Everybody' by DON BROCO removed for explicit rating.
Track 'Gumshield' by DON BROCO added to sample pool.
Track 'Come out to La' by DON BROCO added to sample pool.
Track '

KeyboardInterrupt: 

In [9]:
# Initial setup
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=c_id, # Import spotify authentication
                                            client_secret=s_id,
                                            redirect_uri="http://localhost:1410/",
                                            scope=("user-library-read","playlist-modify-private")))

outDir = 'Resources/'
os.makedirs(outDir, exist_ok = True)
filepath = getGoogleSheet(s_csv, outDir, "SpotiFire.csv")


os.makedirs(outDir, exist_ok = True)
filepath = getGoogleSheet(ss_csv, outDir, "SpotiFire_Schedule.csv")


# Define me
me = my_id

# Playlist id's
playlist1_uid=p1_uid
playlist2_uid=p2_uid
playlist3_uid=p3_uid
playlist4_uid=p4_uid
playlist_combo_uid=pc_uid
playlist_combo2_uid=pc2_uid

# Extract date from calender
today = date.today()
today = str(today)
year, month, day = map(int, today.split("-"))
today = f"{year},{month},{day}"
print(f"Creating playlist for {today}")

# Manual entry
# today = '2024,6,4'

# Import schedule
schedule = pd.read_csv('Resources/SpotiFire_Schedule.csv')

# Extract todays schedule
today_schedule = schedule.loc[schedule['Date'] == today]
today_schedule = today_schedule.reset_index(drop=True)

line_list = []
line_schedules = []

# Break schedule into lines
line_list = []
line_schedules = []

for line_num in range(1, 5):
    line_schedule = today_schedule.loc[0, [f'Line {line_num} ({i})' for i in range(1, 10)]].values.tolist()
    line_schedules.append(line_schedule)
    line_list.extend(line_schedule)

holiday_key_words = ['Christmas','christmas','Santa','santa']

# Import music choices
choices = pd.read_csv("Resources/SpotiFire.csv")

# Lists to hold artists and songs
artists = []
songs = []
line1_songs = []
line2_songs = []
line3_songs = []
line4_songs = []

# Loop to collect songs for each formulator
for i in range(len(line_list)):
    artists = [] # Reset artist list
    songs = [] # Reset song list
    formulator_choices = choices.loc[line_list[i] == choices['id']] # Select row from spotifire where ID matches the schedule
    print("--------------------------------------------------------------------")
    print("--------------------------------------------------------------------")
    print(f"Collecting artists for {line_list[i]}")
    
    # Check if cell is empty in spotifire_schedule
    if formulator_choices.empty:
        continue  # Skip to the next iteration if it's empty
    
    # Select up to ten artists from formulator's row
    for t in range(10):
        artists.append(formulator_choices.iloc[0,t+1])
        print(f"Artist choice {t}:{formulator_choices.iloc[0,t+1]}")
    
    # Clean the artist list
    artists_clean = pd.DataFrame(artists) # Convert to dataframe
    artists_clean = artists_clean.dropna() # Drop nan
    artists_clean = artists_clean.reset_index(drop=True) # Reset index

    # Loop through artist list
    for t in range(len(artists_clean)):
        artist_name = artists_clean[0][t]
        search_results = sp.search(q='artist:' + artist_name, type='artist', limit=1)
        if search_results['artists']['total'] == 0: # If no artists found
            print(f"No results for {artist_name}") # Print message
            continue # Skip
        print(f"Searching {artist_name}'s related artists")
        time.sleep(6)

        # Collect URI's
        artist_uri = search_results['artists']['items'][0]['uri']
        related_artist_search_results = sp.artist_related_artists(artist_uri) # Search related artists
        related_artist_uri1 = related_artist_search_results['artists'][0]['uri']
        related_artist_uri2 = related_artist_search_results['artists'][1]['uri']
        related_artist_uri3 = related_artist_search_results['artists'][2]['uri']

        # Collect top tracks for each artist
        artist_top_songs = sp.artist_top_tracks(artist_uri,country='US')
        related_artist_top_songs1 = sp.artist_top_tracks(related_artist_uri1, country='US')
        print(f'Related artist 1: {related_artist_search_results["artists"][0]["name"]}')
        time.sleep(6)
        related_artist_top_songs2 = sp.artist_top_tracks(related_artist_uri2, country='US')
        print(f'Related artist 2: {related_artist_search_results["artists"][1]["name"]}')
        time.sleep(6)
        related_artist_top_songs3 = sp.artist_top_tracks(related_artist_uri3, country='US')
        print(f'Related artist 3: {related_artist_search_results["artists"][2]["name"]}')
        time.sleep(6)

        # Randomly select one song from each related artist
        related_artist_top_songs_sample1 = random.sample(related_artist_top_songs1['tracks'],1)
        related_artist_top_songs_sample2 = random.sample(related_artist_top_songs2['tracks'],1)
        related_artist_top_songs_sample3 = random.sample(related_artist_top_songs3['tracks'],1)

        sample_lists = [related_artist_top_songs_sample1, related_artist_top_songs_sample2, related_artist_top_songs_sample3]

        for sample in sample_lists:
            for j in range(len(sample)):
                if sample[j]['name'] in holiday_key_words:
                    print(f"Track '{sample[j]['name']}' by {sample[j]['artists'][0]['name']} removed for Holiday status.")
                    continue
                if sample[j]['explicit'] == False:
                    print(f"Track '{sample[j]['name']}' by {sample[j]['artists'][0]['name']} added to sample pool.")
                    songs.append(sample[j]['uri'])
                else:
                    print(f"Track '{sample[j]['name']}' by {sample[j]['artists'][0]['name']} removed for explicit rating.")

        for j in range(len(artist_top_songs['tracks'])):
            if artist_top_songs['tracks'][j]['name'] in holiday_key_words:
                print(f"Track '{artist_top_songs['tracks'][j]['name']}' by {artist_top_songs['tracks'][j]['artists'][0]['name']} removed for Holiday status.")
                continue
            if artist_top_songs['tracks'][j]['explicit'] == False:
                print(f"Track '{artist_top_songs['tracks'][j]['name']}' by {artist_top_songs['tracks'][j]['artists'][0]['name']} added to sample pool.")
                songs.append(artist_top_songs['tracks'][j]['uri'])
            else:
                print(f"Track '{artist_top_songs['tracks'][j]['name']}' by {artist_top_songs['tracks'][j]['artists'][0]['name']} removed for explicit rating.")


    # If song count is below 30
    if len(songs) < 30:
        songs_sample = random.sample(songs, len(songs))
        if len(songs) > 2:
            filler_tracks = random.sample(related_artist_top_songs1['tracks'],3) # Grab 3 filler tracks from related artist 1
            for j in range(len(filler_tracks)):
                if filler_tracks[j]['explicit'] == False: # Check if songs are explicit
                    songs_sample.append(filler_tracks[j]['uri'])
                    print(f"Track '{filler_tracks[j]['name']}' by {filler_tracks[j]['artists'][0]['name']} added to sample pool.")
                else:
                    print(f"Track '{filler_tracks[j]['name']}' by {filler_tracks[j]['artists'][0]['name']} removed for explicit rating.")
                    
    # If song count is 30 or greater, take a random sample of 30 songs and add to playlist         
    else: 
        songs_sample = random.sample(songs,30)
    if line_list[i] in line_schedules[0]:
        line1_songs.extend(songs_sample)
        print(f"Formulator {line_list[i]} is adding {len(songs_sample)} songs to line 1")
    if line_list[i] in line_schedules[1]:
        line2_songs.extend(songs_sample)
        print(f"Formulator {line_list[i]} is adding {len(songs_sample)} songs to line 2")
    if line_list[i] in line_schedules[2]:
        line3_songs.extend(songs_sample)
        print(f"Formulator {line_list[i]} is adding {len(songs_sample)} songs to line 3")
    if line_list[i] in line_schedules[3]:
        line4_songs.extend(songs_sample)
        print(f"Formulator {line_list[i]} is adding {len(songs_sample)} songs to line 4")


# Remove duplicates
line1_songs = list(set(line1_songs))
line2_songs = list(set(line2_songs))
line3_songs = list(set(line3_songs))
line4_songs = list(set(line4_songs))

CSV file saved to: Resources/SpotiFire.csv
CSV file saved to: Resources/SpotiFire_Schedule.csv
Creating playlist for 2024,6,8
--------------------------------------------------------------------
--------------------------------------------------------------------
Artist choice 0:Celtic Woman
Artist choice 1:Red Hot Chili Peppers
Artist choice 2:Tame Impala
Artist choice 3:nan
Artist choice 4:nan
Artist choice 5:nan
Artist choice 6:nan
Artist choice 7:nan
Artist choice 8:nan
Artist choice 9:nan
Searching Celtic Woman's related artists
Related artist 1: Celtic Thunder
Related artist 2: Chloe Agnew
Related artist 3: Lisa Kelly
No holiday here
Track 'Amazing Grace' by Celtic Thunder added to sample pool.
No holiday here
Track 'Vivaldi's 'Rain'' by Chloe Agnew added to sample pool.
No holiday here
Track 'May It Be - From Lord of the Rings' by Lisa Kelly added to sample pool.
Track 'Teir Abhaile Riu' by Celtic Woman added to sample pool.
Track 'You Raise Me Up' by Celtic Woman added to sampl

KeyboardInterrupt: 